In [1]:
import requests
import pandas as pd

BASE_URL = "https://www.cheapshark.com/api/1.0/deals"

def fetch_all_deals(store_id="1", page_size=60, max_pages=30):
    """
    Fetch multiple pages of CheapShark deals.

    - store_id="1" means Steam
    - page_size=60 is the maximum number of deals per page
    - max_pages is a safety limit so we don't loop forever
    """
    all_deals = []
    page = 0

    while page < max_pages:
        params = {
            "storeID": store_id,
            "pageSize": page_size,
            "pageNumber": page,
            "sortBy": "Savings",  # sort by biggest discount first
        }
        print(f"Requesting page {page} ...")
        resp = requests.get(BASE_URL, params=params, timeout=30)

        if resp.status_code != 200:
            print(f"  ❌ Status code {resp.status_code}, stopping.")
            break

        deals = resp.json()
        print(f"  Got {len(deals)} deals.")

        if not deals:
            # No more data, we've reached the end
            break

        all_deals.extend(deals)

        # If we got fewer than page_size deals, this is probably the last page
        if len(deals) < page_size:
            break

        page += 1

    print(f"\nTotal deals fetched: {len(all_deals)}")

    df = pd.DataFrame(all_deals)

    # Convert CheapShark's releaseDate (Unix seconds) to datetime, year, and a readable date string
    if "releaseDate" in df.columns:
        df["release_date"] = pd.to_datetime(
            df["releaseDate"],
            unit="s",
            utc=True,
            errors="coerce"
        )
        df["release_year"] = df["release_date"].dt.year

        # 🔹 New column: formatted date string (e.g. 01/01/70, 10/04/16, etc.)
        df["date"] = df["release_date"].dt.strftime("%m/%d/%y")

    return df


df_raw = fetch_all_deals()
df_raw.head()


Requesting page 0 ...


  Got 60 deals.
Requesting page 1 ...


  Got 60 deals.
Requesting page 2 ...


  Got 60 deals.
Requesting page 3 ...


  Got 60 deals.
Requesting page 4 ...


...

  Got 60 deals.
Requesting page 27 ...


  Got 60 deals.
Requesting page 28 ...


  Got 60 deals.
Requesting page 29 ...


  Got 60 deals.

Total deals fetched: 1800


,internalName,title,metacriticLink,dealID,storeID,gameID,salePrice,normalPrice,isOnSale,savings,...,steamRatingPercent,steamRatingCount,steamAppID,releaseDate,lastChange,dealRating,thumb,release_date,release_year,date
0,GRAVNEWTON,GravNewton,/game/gravnewton/,S%2BYmz0CXz%2F0%2BzQexUe3m3RCEyTlqLkWVm8rSZ9Rm...,1,171344,0.49,9.99,1,95.095095,...,78,375,549100,1501027200,1766094583,2.8,https://shared.fastly.steamstatic.com/store_it...,2017-07-26 00:00:00+00:00,2017,07/26/17
1,EVOLUSHIP,EvoluShip,/game/evoluship/,tPSU54Xtk5qDpV%2FiBH%2FqbSLCpeB5Unmln7mkOpUqT%...,1,253747,0.49,9.99,1,95.095095,...,70,40,1937180,1669075200,1766094809,0.0,https://shared.fastly.steamstatic.com/store_it...,2022-11-22 00:00:00+00:00,2022,11/22/22
2,ASTEROIDCORE,Asteroid Core,None,qujpOGdMsg0topytx6Kn2HyLiMgVEfvliWy8%2F0MEf0s%3D,1,288906,0.49,9.99,1,95.095095,...,66,30,2608120,0,1766098457,0.0,https://shared.fastly.steamstatic.com/store_it...,1970-01-01 00:00:00+00:00,1970,01/01/70
3,BAKERYSIMULATOR,Bakery Simulator,/game/bakery-simulator/,CgQ3bEtWuNSfva1MNESmK0voFRJXFy4TRQv1KMlKZA8%3D,1,245267,0.79,15.99,1,95.059412,...,66,1961,1031120,1651536000,1766088015,6.3,https://shared.fastly.steamstatic.com/store_it...,2022-05-03 00:00:00+00:00,2022,05/03/22
4,LIVINGFOREST,LivingForest,/game/livingforest/,EumHkrN1aqGgTfFRb50XAJgQW0KiyyjN8pLT4VGeGSw%3D,1,312087,0.79,15.99,1,95.059412,...,0,0,3027490,1730505600,1766118691,2.1,https://shared.fastly.steamstatic.com/store_it...,2024-11-02 00:00:00+00:00,2024,11/02/24


In [2]:
df_raw.to_csv("cheapshark_updated.csv", index=False)


In [3]:
# Create a smaller table for the range/arrow plot: top 20 deals by dealRating
df_top20 = (
    df_raw.copy()
    .assign(dealRating_float=pd.to_numeric(df_raw["dealRating"], errors="coerce"))
    .sort_values("dealRating_float", ascending=False)
    .head(20)
)

# Keep only the columns needed for Datawrapper
df_top20_for_chart = df_top20[["title", "normalPrice", "salePrice", "release_year"]]

df_top20_for_chart.head()


,title,normalPrice,salePrice,release_year
294,SteamWorld Dig 2,19.99,1.19,2017
800,LIMBO,9.99,0.99,2011
1759,INSIDE,19.99,1.99,2016
779,Hitman 2: Silent Assassin,8.99,0.89,2002
785,Bulb Boy,8.99,0.89,2015


In [4]:
df_top20_for_chart.to_csv("steam_deals_top20_rangeplot.csv", index=False)
